In [13]:
import pandas as pd

In [42]:
data_firm_level = pd.read_stata("../Data/Firm_patent/data_firm_level.dta")

In [43]:
data_firm_level.head()

,year,gvkey,fyr,csho,ebit,ebitda,emp,pstk,sale,xrd,...,rivao,rintan,ract,value,value_e,value_d,qkstock,tobinq,tobinq_e,tobinq_d
0,1994.0,10846,12.0,363.661,3856.000,4977.000,304.000,NaN,45419.000,831.000,...,148.300720,0.000000,15795.056641,NaN,NaN,-10743.563477,19477.857422,NaN,NaN,-0.551578
1,1995.0,10846,12.0,363.800,3986.000,5242.000,308.000,NaN,49732.000,923.000,...,169.000000,0.000000,16314.000000,NaN,NaN,-11067.000000,20419.000000,NaN,NaN,-0.541995
2,1996.0,10846,12.0,363.816,4473.000,5794.000,306.000,153.435,52161.000,934.000,...,187.683273,0.000000,16892.472656,51197.640625,62475.253906,-11277.615234,19883.673828,2.574858,3.142038,-0.567180
3,1996.0,153351,12.0,15.669,-3.810,-3.766,0.012,0.001,2.000,3.344,...,0.000000,0.000000,47.806450,124.507561,172.314011,-47.806450,0.085044,1464.037354,2026.175293,-562.137939
4,1996.0,27961,12.0,NaN,15.514,20.698,0.915,0.000,170.821,0.900,...,0.000000,144.434006,57.626587,NaN,NaN,98.776138,204.719437,NaN,NaN,0.482495


In [ ]:
data_firm_level.describe()

In [44]:
patents_firm_merge = pd.read_stata(r"..\Data\Firm_patent\patents_firm_merge.dta")

In [46]:
feature_names = [c for c in patents_firm_merge.columns if c not in ["index", "patnum", "fdate", "idate", "pdate", "permno", "year", "Npats", "_merge"]]

In [47]:
df = patents_firm_merge[feature_names]

In [48]:
df.dtypes

patent_class     object
subclass         object
ncites          float64
xi              float64
Tcw             float64
Tsm             float64
tcw             float64
tsm             float64
dtype: object

In [49]:
df.astype({"patent_class": float, "subclass": float})

ValueError: could not convert string to float: '206000O'

* Lisser / au temps
* Passer au log
* Enlever les index
* Normaliser
* ACP
* km.cover(n = 20, cov = 0.5)
* km.map(ACP, data, cover)
* Clustering
* Créer le graph

In [45]:
patents_firm_merge.iloc[20000:20010]

,index,patnum,fdate,idate,pdate,permno,patent_class,subclass,ncites,xi,year,Npats,Tcw,Tsm,tcw,tsm,_merge
20000,5703160,7289102,06/06/2001,10/30/2007,02/21/2002,10107.0,345.0,156000O,11.0,48.29374,2007,1632,4798.287629,61577.590671,0.075957,0.974776,matched (3)
20001,5698136,7284056,02/13/2002,10/16/2007,04/10/2003,10107.0,709.0,227000O,3.0,20.73327,2007,1632,4798.287629,61577.590671,0.075957,0.974776,matched (3)
20002,5657515,7243239,06/28/2002,07/10/2007,01/15/2004,10107.0,713.0,184000O,4.0,38.78856,2007,1632,4798.287629,61577.590671,0.075957,0.974776,matched (3)
20003,5722402,7308408,09/29/2004,12/11/2007,,10107.0,704.0,266000O,4.0,50.25811,2007,1632,4798.287629,61577.590671,0.075957,0.974776,matched (3)
20004,5698282,7284202,02/14/2000,10/16/2007,,10107.0,715.0,744000O,10.0,20.73327,2007,1632,4798.287629,61577.590671,0.075957,0.974776,matched (3)
20005,5701173,7287106,12/15/2004,10/23/2007,05/05/2005,10107.0,710.0,057000O,1.0,31.18564,2007,1632,4798.287629,61577.590671,0.075957,0.974776,matched (3)
20006,5686691,7272557,05/01/2003,09/18/2007,11/04/2004,10107.0,704.0,230000O,1.0,44.12155,2007,1632,4798.287629,61577.590671,0.075957,0.974776,matched (3)
20007,5713368,7299351,09/19/2001,11/20/2007,03/20/2003,10107.0,713.0,156000O,13.0,45.34085,2007,1632,4798.287629,61577.590671,0.075957,0.974776,matched (3)
20008,5707053,7293013,10/19/2004,11/06/2007,,10107.0,707.0,694000O,5.0,21.28750,2007,1632,4798.287629,61577.590671,0.075957,0.974776,matched (3)
20009,5707086,7293046,12/23/2004,11/06/2007,06/30/2005,10107.0,707.0,613000O,0.0,21.28750,2007,1632,4798.287629,61577.590671,0.075957,0.974776,matched (3)


In [15]:
patents_firm_merge.columns

Index(['index', 'patnum', 'fdate', 'idate', 'pdate', 'permno', 'patent_class',
       'subclass', 'ncites', 'xi', 'year', 'Npats', 'Tcw', 'Tsm', 'tcw', 'tsm',
       '_merge'],
      dtype='object')

In [10]:
patents_firm_merge.count()

index           1844881
patnum          1844881
fdate           1844881
idate           1844881
pdate           1844881
permno          1844881
patent_class    1844881
subclass        1844881
ncites          1801879
xi              1801879
year            1844881
Npats           1844881
Tcw             1844881
Tsm             1844881
tcw             1669418
tsm             1669418
_merge          1844881
dtype: int64

In [15]:
patents_firm_merge["permno"].value_counts()

12490.0    72500
12060.0    61116
64231.0    51964
21152.0    40937
53727.0    37800
           ...  
18200.0        1
50024.0        1
80248.0        1
80235.0        1
69243.0        1
Name: permno, Length: 6995, dtype: int64

In [ ]:
data_patent_level = pd.read_stata(r"..\Data\Patent_level_data\data_patent_level.dta")

In [ ]:
data_patent_level.head()

In [ ]:
data_patent_level.describe()

In [16]:
data_patent_level = pd.read_stata(r"..\Data\Patent_level_data\data_patent_level.dta")

In [17]:
data_patent_level.head()

,patnum,fdate,idate,pdate,permno,patent_class,subclass,ncites,xi,year,Npats,Tcw,Tsm,forward_citations,backward_citations
0,1568124,06/12/1920,01/05/1926,,12503.0,56.0,006000O,0.0,0.161681,1926,15,25.664662,4.485454,NaN,NaN
1,1568134,09/09/1922,01/05/1926,,12503.0,239.0,584000O,0.0,0.161681,1926,15,25.664662,4.485454,NaN,NaN
2,1568142,08/13/1920,01/05/1926,,10401.0,333.0,168000O,1.0,1.625674,1926,430,871.180249,187.894016,NaN,1.0
3,1568163,05/03/1920,01/05/1926,,12503.0,56.0,011500O,1.0,0.161681,1926,15,25.664662,4.485454,NaN,NaN
4,1568338,11/02/1923,01/05/1926,,14875.0,400.0,689000O,3.0,0.163435,1926,104,134.871391,5.036761,NaN,3.0


In [ ]:
data_patent_level.describe()

In [ ]:
cites = pd.read_stata(r"..\Data\Patent_level_data\USPatent_1926-2010\cites\cites.dta")

In [ ]:
cites.head()

In [ ]:
cites.describe()

In [ ]:
firm_innovation_v2 = pd.read_stata(r"..\Data\Patent_level_data\USPatent_1926-2010\firm_innovation\firm_innovation_v2.dta")

In [ ]:
firm_innovation_v2.head()

In [ ]:
firm_innovation_v2.describe()

In [ ]:
patents_xi = pd.read_stata(r"..\Data\Patent_level_data\USPatent_1926-2010\patents_xi\patents_xi.dta")

In [ ]:
patents_xi.head()

In [ ]:
patents_xi.describe()

In [ ]:
patent_values = pd.read_stata(r"..\Data\Patent_level_data\Patent_CRSP_match_1929-2017\patent_values\patent_values.dta")

In [ ]:
patent_values.head()

In [ ]:
patent_values.describe()

In [11]:
# Import the class
import kmapper as km
from kmapper import jupyter # Creates custom CSS full-size Jupyter screen

# Some sample data
from sklearn import datasets
data, labels = datasets.make_circles(n_samples=5000, noise=0.03, factor=0.3)

# Initialize
mapper = km.KeplerMapper(verbose=1)

# Fit to and transform the data
projected_data = mapper.fit_transform(data, projection=[0,1]) # X-Y axis

# Create dictionary called 'graph' with nodes, edges and meta-information
graph = mapper.map(projected_data, data)

# Visualize it
html = mapper.visualize(graph, path_html="make_circles_keplermapper_output.html",
                 title="make_circles(n_samples=5000, noise=0.03, factor=0.3)")

# Inline display
# jupyter.display(path_html="http://mlwave.github.io/tda/word2vec-gender-bias.html")
jupyter.display(path_html="make_circles_keplermapper_output.html")

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: [0, 1]
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (5000, 2)

..Projecting data using: [0, 1]

..Scaling with: MinMaxScaler()

Mapping on data shaped (5000, 2) using lens shaped (5000, 2)

Creating 100 hypercubes.

Created 93 edges and 57 nodes in 0:00:00.135998.
Wrote visualization to: make_circles_keplermapper_output.html


In [6]:
data


array([[ 0.20432314,  0.22339408],
       [-0.89852117,  0.54866288],
       [ 0.31528727,  0.02173862],
       ...,
       [ 0.50171767,  0.81474048],
       [ 0.22024521, -0.19627437],
       [-0.70816586,  0.68560701]])

In [7]:
projected_data

array([[0.59212259, 0.60168439],
       [0.07926769, 0.75177098],
       [0.64372416, 0.50863587],
       ...,
       [0.73041975, 0.87454537],
       [0.59952682, 0.4080396 ],
       [0.16778849, 0.81496018]])